In [4]:
# AdaBoost 모델 생성 및 저장
import pandas as pd
import numpy as np
import joblib
import os

from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier


# Tree 전처리 데이터 로드
tree_data = pd.read_csv('../data/netflix_customer_churn_tree_preprocessed.csv')

X_tune = tree_data.drop(columns=['churned'])
y_tune = tree_data['churned']

# Train/Test Split
X_train_tune, X_test_tune, y_train_tune, y_test_tune = train_test_split(
    X_tune, y_tune, test_size=0.2, random_state=42
)

# Parameters
adaboost_params = {
    'estimator__max_depth': 4,
    'learning_rate': 0.05,
    'n_estimators': 250
}

print("=== AdaBoost 모델 생성 및 저장 ===\n")
print(f"Parameters:")
for param, value in adaboost_params.items():
    print(f"  • {param}: {value}")

# AdaBoost 모델 생성
adaboost_model = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=4, class_weight='balanced', criterion='gini', splitter='best', random_state=42),
    n_estimators=adaboost_params['n_estimators'],
    learning_rate=adaboost_params['learning_rate'],
    random_state=42
)

# 모델 학습
print("\n모델 학습 중...")
adaboost_model.fit(X_train_tune, y_train_tune)

# 모델 평가
y_pred_train = adaboost_model.predict(X_train_tune)
y_pred_test = adaboost_model.predict(X_test_tune)

train_acc = accuracy_score(y_train_tune, y_pred_train)
test_acc = accuracy_score(y_test_tune, y_pred_test)

print(f"\n성능 평가:")
print(f"  Train Accuracy: {train_acc:.4f}")
print(f"  Test Accuracy: {test_acc:.4f}")
print(f"  Overfitting: {(train_acc - test_acc):.4f}")

print(f"\nTest Classification Report:")
print(classification_report(y_test_tune, y_pred_test, target_names=['Not Churned', 'Churned']))

# 모델 저장 디렉토리 생성
model_dir = './models'
os.makedirs(model_dir, exist_ok=True)

# 모델 저장 (.pth 형식)
model_filename = os.path.join(model_dir, 'adaboost_model.pth')
joblib.dump(adaboost_model, model_filename)
print(f"\n✅ 모델이 저장되었습니다: {model_filename}")

=== AdaBoost 모델 생성 및 저장 ===

Parameters:
  • estimator__max_depth: 4
  • learning_rate: 0.05
  • n_estimators: 250

모델 학습 중...

성능 평가:
  Train Accuracy: 0.9487
  Test Accuracy: 0.9370
  Overfitting: 0.0117

Test Classification Report:
              precision    recall  f1-score   support

 Not Churned       0.91      0.97      0.94       498
     Churned       0.97      0.90      0.93       502

    accuracy                           0.94      1000
   macro avg       0.94      0.94      0.94      1000
weighted avg       0.94      0.94      0.94      1000


✅ 모델이 저장되었습니다: ./models\adaboost_model.pth


In [3]:
print("새로운 고객 데이터를 입력해주세요 :")

# Label Encoding 맵 정의 (데이터 전처리 시 사용된 실제 인코딩 순서에 따라 다를 수 있습니다)
# 이 맵은 사용자 입력 편의를 위해 코드에 직접 명시합니다.
gender_map = {0: 'Female', 1: 'Male', 2: 'Other'}
subscription_type_map = {0: 'Basic', 1: 'Standard', 2: 'Premium'}
region_map = {0: 'Africa', 1: 'Asia', 2: 'Europe', 3: 'North America', 4: 'Oceania', 5: 'South America'}
device_map = {0: 'Desktop', 1: 'Laptop', 2: 'Mobile', 3: 'TV', 4: 'Tablet'}
favorite_genre_map = {0: 'Action', 1: 'Comedy', 2: 'Documentary', 3: 'Drama', 4: 'Horror', 5: 'Romance', 6: 'Sci-Fi'}


# 인코딩 맵 정보를 포함한 입력 프롬프트 생성 함수
def create_input_prompt(feature_name, encoding_map):
    options = ", ".join([f"{key}: {value}" for key, value in encoding_map.items()])
    return f"{feature_name} ({options}): "

# --- 모델 로드 ---
import joblib # scikit-learn 모델 로드를 위해 joblib 사용

model_path = './models/adaboost_model.pth' # 모델 파일 경로 지정

try:
    # joblib을 사용하여 모델 로드
    loaded_model = joblib.load(model_path)

    # 모델이 유효한 scikit-learn 분류 모델인지 확인
    if not hasattr(loaded_model, 'predict') or not hasattr(loaded_model, 'predict_proba'):
         raise TypeError("로드된 파일이 유효한 scikit-learn 분류 모델이 아닌 것 같습니다.")

    print(f"\n✅ 모델 로딩 완료: {model_path}")

except FileNotFoundError:
    print(f"\n⚠️ 오류: 모델 파일 '{model_path}'을 찾을 수 없습니다.")
    loaded_model = None # 모델 로드 실패 시 None으로 설정
except Exception as e:
    print(f"\n⚠️ 모델 로딩 중 오류 발생: {e}")
    loaded_model = None # 모델 로드 실패 시 None으로 설정


try:
    if loaded_model is None:
        print("\n모델 로드에 실패하여 예측을 수행할 수 없습니다.")
    else:
        age = int(input("나이 : "))
        watch_hours = float(input("시청 시간 : "))
        last_login_days = int(input("마지막 로그인한지 얼마나됨 ㅋ : "))
        number_of_profiles = int(input("프로필 개수 : "))

        # 인코딩 맵 정보를 포함한 프롬프트로 입력 받기
        gender_encoded = int(input(create_input_prompt("성별 ", gender_map)))
        subscription_type_encoded = int(input(create_input_prompt("구독 타입 ", subscription_type_map)))
        region_encoded = int(input(create_input_prompt("지역 ", region_map)))
        device_encoded = int(input(create_input_prompt("기기 ", device_map)))
        favorite_genre_encoded = int(input(create_input_prompt("선호 장르 ", favorite_genre_map)))

        new_customer_data = {
            'age': [age],
            'watch_hours': [watch_hours],
            'last_login_days': [last_login_days],
            'number_of_profiles': [number_of_profiles],
            'gender_encoded': [gender_encoded],
            'subscription_type_encoded': [subscription_type_encoded],
            'region_encoded': [region_encoded],
            'device_encoded': [device_encoded],
            'favorite_genre_encoded': [favorite_genre_encoded]
        }

        new_customer_df = pd.DataFrame(new_customer_data)

        # --- 로드된 모델로 예측 수행 ---
        # scikit-learn 모델은 numpy 배열 입력을 기대합니다.
        # DataFrame을 numpy 배열로 변환
        new_customer_array = new_customer_df.values

        # 예측된 클래스
        predicted_churn = loaded_model.predict(new_customer_array)

        # 예측 확률
        predicted_proba = loaded_model.predict_proba(new_customer_array)


        # 예측 결과 출력
        print("\n--- 예측 결과 ---")
        if predicted_churn[0] == 1:
            print("예측 결과: 고객 이탈 가능성이 높습니다 ")
        else:
            print("예측 결과: 고객 이탈 가능성이 낮습니다 ")

        # 이탈 확률 (클래스 1의 확률) 출력
        print(f"이탈 예측 확률 : {predicted_proba[0][1].round(3) * 100}%")


except ValueError:
    print("\n⚠️ 잘못된 입력 형식입니다. 숫자를 입력해야 하는 항목에 숫자가 아닌 값이 입력되었습니다.")
except Exception as e:
    print(f"\n⚠️ 예측 중 오류가 발생했습니다: {e}")


새로운 고객 데이터를 입력해주세요 :

✅ 모델 로딩 완료: ./models/adaboost_model.pth

--- 예측 결과 ---
예측 결과: 고객 이탈 가능성이 낮습니다 
이탈 예측 확률 : 40.0%


c:\SKN20-2nd-5TEAM\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(
c:\SKN20-2nd-5TEAM\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(


# 입력받은 고객정보를 바탕으로 이탈안하는 애들이랑 가장 가까운값 찾아서 걔네 평균으로 구독타입 추천

In [4]:
non_churned_customers = tree_data[tree_data['churned'] == 0].copy()
new_customer_data = {
    'age': [age],
    'watch_hours': [watch_hours],
    'last_login_days': [last_login_days],
    'number_of_profiles': [number_of_profiles],
    'gender_encoded': [gender_encoded],
    'subscription_type_encoded': [subscription_type_encoded],
    'region_encoded': [region_encoded],
    'device_encoded': [device_encoded],
    'favorite_genre_encoded': [favorite_genre_encoded]
}

new_customer_df = pd.DataFrame(new_customer_data)
from scipy.spatial.distance import cdist
feature_cols = ['age', 'watch_hours', 'last_login_days', 'number_of_profiles',
                'gender_encoded', 'subscription_type_encoded', 'region_encoded',
                'device_encoded', 'favorite_genre_encoded']
new_customer_features = new_customer_df[feature_cols]
non_churned_features = non_churned_customers[feature_cols]
distances = cdist(new_customer_features, non_churned_features, metric='euclidean')
distances = distances[0]
non_churned_customers['distance_to_new_customer'] = distances
K = 5       #### 이 만큼의 수의 가까운 데이터들 찾아서 평균 냄
nearest_neighbors = non_churned_customers.sort_values(by='distance_to_new_customer').head(K)
nearest_subscription_types = nearest_neighbors['subscription_type_encoded']
recommended_subscription_encoded = nearest_subscription_types.mode()[0]
recommended_subscription_type = subscription_type_map[recommended_subscription_encoded]
print(f"\n--- 추천 구독 타입 ---")
print(f"\n추천 구독 타입: {recommended_subscription_type}")


--- 추천 구독 타입 ---

추천 구독 타입: Premium
